In [51]:
# imports

import os
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from report_writer import ReportWriter
from utils import get_info

In [52]:
RESULTS_FOLDER = "../results"
DATA_FOLDER = "../compas-analysis"

In [54]:
# load data file located in compass-analysis folder

data_file_path = os.path.join(DATA_FOLDER, 'compas-scores-two-years.csv')
df = pd.read_csv(data_file_path)
print(f'Shape fo dataset: {df.shape}')

Shape fo dataset: (7214, 53)


## First look at the data

We look at the dataset, storing all the results in excel. in this step we will
- look at the dataframe information
- capture the first 5 rows
- look at information regarding numerical fields, min. max etc


In [55]:

report_file_path = os.path.join(RESULTS_FOLDER, 'initial_analysis_report.xlsx')

rep_writer = ReportWriter(report_file_path)

# what are the data types
rep_writer.add_data_frame(get_info(df), 'data_types')

# look at first 5 rows in dataset
rep_writer.add_data_frame(df.head(), 'first_5_rows')

# summerize numerical vals
rep_writer.add_data_frame(df.describe(), 'numerical_summary')

rep_writer.save()

In [14]:
df.isnull().sum()

id                            0
name                          0
first                         0
last                          0
compas_screening_date         0
sex                           0
dob                           0
age                           0
age_cat                       0
race                          0
juv_fel_count                 0
decile_score                  0
juv_misd_count                0
juv_other_count               0
priors_count                  0
days_b_screening_arrest     307
c_jail_in                   307
c_jail_out                  307
c_case_number                22
c_offense_date             1159
c_arrest_date              6077
c_days_from_compas           22
c_charge_degree               0
c_charge_desc                29
is_recid                      0
r_case_number              3743
r_charge_degree            3743
r_days_from_arrest         4898
r_offense_date             3743
r_charge_desc              3801
r_jail_in                  4898
r_jail_o

## Missing Values Handling

In [17]:
# Check for missing values
print("Missing Values:\n", df.isnull().sum())

# Drop rows with missing values (if critical features have missing data)
df_isolated = df.dropna()

# fill with median
# df_isolated.fillna(df_isolated.median(), inplace=True)

# Check for missing values
print("Missing Values:\n", df.isnull().sum())



Missing Values:
 id                            0
name                          0
first                         0
last                          0
compas_screening_date         0
sex                           0
dob                           0
age                           0
age_cat                       0
race                          0
juv_fel_count                 0
decile_score                  0
juv_misd_count                0
juv_other_count               0
priors_count                  0
days_b_screening_arrest     307
c_jail_in                   307
c_jail_out                  307
c_case_number                22
c_offense_date             1159
c_arrest_date              6077
c_days_from_compas           22
c_charge_degree               0
c_charge_desc                29
is_recid                      0
r_case_number              3743
r_charge_degree            3743
r_days_from_arrest         4898
r_offense_date             3743
r_charge_desc              3801
r_jail_in              

In [902]:
# Features to isolate
features_to_isolate = [
    'age',                   # Current age
    'priors_count',          # Total prior offenses
    'juv_fel_count',         # Juvenile felony offenses
    'juv_misd_count',        # Juvenile misdemeanors
    'juv_other_count',       # Other juvenile offenses
    # 'violent_recid',         # Indicator of violent recidivism - removed as all data is null
    'days_b_screening_arrest', # Days between arrest and screening
    'is_recid',              # General recidivism indicator (optional for context)
    'c_charge_desc'          # Charge description (optional for text analysis)
]

# Isolate the features into a new dataset
df_isolated = df[features_to_isolate]

print(f'Shape of dataset: {df_isolated.shape}')

# Save
df_isolated.to_csv("isolated_features.csv", index=False)

Shape of dataset: (7214, 8)


## Data Preprocessing

We now start preprocessing the data



In [10]:
categorical_columns = df_isolated.select_dtypes(include=['object']).columns.tolist()

print("Categorical features in this dataset:\n", categorical_columns)

for categorical_column in categorical_columns:
    print(f'For the category: {categorical_column}')
    print(df_isolated[categorical_column].unique())



NameError: name 'df_isolated' is not defined

In [905]:
# Define 10 categories for crime types
crime_types = {
    'charge-theft': ['Burglary', 'Theft', 'Robbery', 'Burgl', 'Burg', 'Carjacking', 'Stolen'],
    'charge-intoxication': ['Possession', 'Possess', 'Poss', 'Cannabis', 'Alcoholic', 'Control Subst', 'Bev In Pub', 'Methylenediox', 'Cocaine'],
    'charge-violent': ['Homicide', 'Manslaughter', 'Battery', 'Assault', 'Abuse', 'Stalking', 'Aggress', 'Mischief', 'Cruelty', 'Resist', 'Disorderly'],
    'charge-domestic': ['Domestic Violence', 'Neglect Child'],
    'charge-fraud': ['Fraud', 'Forge', 'Tamper', 'Eluding', 'False Name', 'Worthless Check', 'Bribery', 'Corrupt', 'Crim Use of Personal ID Info', 'False Ownership Info'],
    'charge-arson': ['Arson'],
    'charge-driving': ['DUI', 'Driving', 'Drivers', 'D.U.I'],
    'charge-weapons': ['Weapons', 'Firearm', 'Explosives', 'Weapon'],
    'charge-morality': ['Indecent Exposure', 'Loitering', 'Prostitution', 'Voyeurism', 'Pornography'],
}



for crime_category, crime_substrings in crime_types.items():

    # substrings joined with OR operator
    pattern = '|'.join(crime_substrings)

    # create the col for the crime category
    df_isolated[crime_category] = df_isolated['c_charge_desc'].str.contains(pattern, case=False, na=False)
    
    # copy the description in copy desc colum
    df_isolated.loc[df_isolated[crime_category], 'c_charge_desc_copy'] = df_isolated['c_charge_desc']
    
    # erase the original description
    df_isolated['c_charge_desc'] = df_isolated['c_charge_desc'].where(~df_isolated[crime_category], other=None)


others_category = 'charge-others'
df_isolated[others_category] = ~(df_isolated['c_charge_desc'].isnull())
df_isolated.loc[df_isolated[others_category], 'c_charge_desc_copy'] = df_isolated['c_charge_desc']
df_isolated['c_charge_desc'] = df_isolated['c_charge_desc'].where(~df_isolated[others_category], other=None)


for crime_cat_col, _ in crime_types.items():
    print(f'Number for category {crime_cat_col} is {df_isolated[crime_cat_col].sum()}')

print(f'Number for category {others_category} is {df_isolated[others_category].sum()}')




Number for category charge-theft is 1064
Number for category charge-intoxication is 1270
Number for category charge-violent is 1979
Number for category charge-domestic is 76
Number for category charge-fraud is 257
Number for category charge-arson is 2
Number for category charge-driving is 709
Number for category charge-weapons is 44
Number for category charge-morality is 35
Number for category charge-others is 1464


In [906]:
# Define expanded crime categories to cover 99% of data
crime_types = {
    # Theft-related crimes
    'Burglary': 'charge-theft',
    'Theft': 'charge-theft',
    'Robbery': 'charge-theft',
    'Shoplifting': 'charge-theft',
    'Burg': 'charge-theft',
    'Larceny': 'charge-theft',
    'Stolen': 'charge-theft',
    'Auto': 'charge-theft',
    'Vehicle': 'charge-theft',

    # Drug-related crimes
    'Possession': 'charge-drugs',
    'Possess': 'charge-drugs',
    'Poss': 'charge-drugs',
    'Cannabis': 'charge-drugs',
    'Marijuana': 'charge-drugs',
    'Meth': 'charge-drugs',
    'Cocaine': 'charge-drugs',
    'Heroin': 'charge-drugs',
    'Alcoholic': 'charge-drugs',
    'Control Subst': 'charge-drugs',
    'Drug': 'charge-drugs',
    'Narcotics': 'charge-drugs',

    # Violent crimes
    'Battery': 'charge-violent',
    'Batt': 'charge-violent',
    'Assault': 'charge-violent',
    'Abuse': 'charge-violent',
    'Aggress': 'charge-violent',
    'Stalking': 'charge-violent',
    'Fight': 'charge-violent',
    'Threat': 'charge-violent',
    'Harm': 'charge-violent',
    
    # Domestic-related crimes
    'Domestic Violence': 'charge-domestic',
    'Child Abuse': 'charge-domestic',
    'Spouse': 'charge-domestic',

    # Financial crimes
    'Fraud': 'charge-financial',
    'Forge': 'charge-financial',
    'Tamper': 'charge-financial',
    'Eluding': 'charge-financial',
    'Unlicensed': 'charge-financial',
    'False Name': 'charge-financial',
    'Identity': 'charge-financial',
    'Credit': 'charge-financial',
    'Embezzle': 'charge-financial',

    # Property-related crimes
    'Arson': 'charge-property',
    'Vandalism': 'charge-property',
    'Trespass': 'charge-property',
    'Damage': 'charge-property',
    'Burgl': 'charge-property',
    'Looting': 'charge-property',

    # Homicide-related crimes
    'Homicide': 'charge-homicide',
    'Manslaughter': 'charge-homicide',
    'Murder': 'charge-homicide',

    # Traffic-related crimes
    'DUI': 'charge-traffic',
    'D.U.I.': 'charge-traffic',
    'Driving': 'charge-traffic',
    'Drivg': 'charge-traffic',
    'Drivers': 'charge-traffic',
    'Reckless': 'charge-traffic',
    'Speeding': 'charge-traffic',
    'Suspended': 'charge-traffic',
    'License': 'charge-traffic',

    # Weapons-related crimes
    'Weapons': 'charge-weapons',
    'Firearm': 'charge-weapons',
    'Explosives': 'charge-weapons',
    'Gun': 'charge-weapons',
    'Knife': 'charge-weapons',

    # Morality-related crimes
    'Indecent Exposure': 'charge-morality',
    'Loitering': 'charge-morality',
    'Prostitution': 'charge-morality',
    'Voyeur': 'charge-morality',
    'Lewd': 'charge-morality',
    'Obscene': 'charge-morality',
    'Pornography': 'charge-morality',

    # Other crimes
    'Viol': 'charge-other',
    'Arrest Case No Charge': 'charge-other',
    'Resisting': 'charge-other',
    'Disorderly': 'charge-other',
    'Obstruction': 'charge-other',
    'Escape': 'charge-other',
    'Contempt': 'charge-other',
    'Violation': 'charge-other',
    'Harassment': 'charge-other',
    'Misc': 'charge-other'
}

# Apply the categorization logic
for crime_substring, crime_category in crime_types.items():
    df_isolated[crime_category] = df_isolated['c_charge_desc'].str.contains(crime_substring, case=False, na=False)
    df_isolated['c_charge_desc_copy'] = df_isolated[df_isolated['c_charge_desc'].str.contains(crime_substring, case=False, na=False)]['c_charge_desc']
    df_isolated['c_charge_desc'] = df_isolated['c_charge_desc'].where(~df_isolated[crime_category], other=None)




# Display remaining uncategorized entries, if any
uncategorized = df_isolated[df_isolated['c_charge_desc'].isnull()]
print(uncategorized.shape)


df_categorized = df_isolated[df_isolated['c_charge_desc'].isnull() == False]

print(df_categorized.shape)

print(df_categorized)


(6900, 25)
(0, 25)
Empty DataFrame
Columns: [age, priors_count, juv_fel_count, juv_misd_count, juv_other_count, days_b_screening_arrest, is_recid, c_charge_desc, charge-theft, c_charge_desc_copy, charge-intoxication, charge-violent, charge-domestic, charge-fraud, charge-arson, charge-driving, charge-weapons, charge-morality, charge-others, charge-drugs, charge-financial, charge-property, charge-homicide, charge-traffic, charge-other]
Index: []
